In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
import pandas as pd

data = pd.read_csv("C:\\Users\\Yash\\Downloads\\tata.csv")

data["Date"] = pd.to_datetime(data["Date"], format="%d-%m-%Y")

print(data)


           Date        Open        High         Low       Close   Adj Close  \
0    1996-10-23    9.881520   10.361560    9.881520   10.361560    2.303619   
1    1996-10-24   10.864058   11.004420   10.389632   10.529995    2.341065   
2    1996-10-25   10.555260   10.886516   10.434548   10.799491    2.400981   
3    1996-10-28   10.838792   10.976348   10.647899   10.704044    2.379761   
4    1996-10-29   10.667550   10.751768   10.445777   10.538417    2.342938   
...         ...         ...         ...         ...         ...         ...   
6728 2023-08-07  119.599998  119.599998  118.000000  118.849998  118.849998   
6729 2023-08-08  119.000000  119.500000  117.949997  118.150002  118.150002   
6730 2023-08-09  118.349998  120.500000  116.900002  120.199997  120.199997   
6731 2023-08-10  120.000000  120.650002  119.250000  120.000000  120.000000   
6732 2023-08-11  120.300003  121.400002  119.150002  120.500000  120.500000   

         Volume        RSI  Target       SMA50     

In [9]:
class TradingEnv(Env):
    def __init__(self, data_file):
        # Initialize the environment
        super().__init__()
        self.data_file = data_file
        self.data = np.genfromtxt(data_file, delimiter=',')
        self.data_len = len(self.data)
        self.current_data_idx = 0
        self.balance = 100000
        self.initial_balance = 100000
        self.shares = 0
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=1, shape=(45, 9))

    def step(self, action):
        # Take an action in the environment
        if action == 0:  # Buy
            if self.balance >= self.data[self.current_data_idx - 1]['Close']:
                self.shares += 1
                self.balance -= self.data[self.current_data_idx - 1]['Close']
        elif action == 1:  # Hold
            pass
        elif action == 2:  # Sell
            if self.shares > 0:
                shares_price = self.shares * self.data[self.current_data_idx]['Close']
                self.balance += shares_price
                self.shares = 0
        else:
            raise ValueError("Invalid action")

        self.current_data_idx += 1
        reward = self.balance - self.initial_balance
        done = self.current_data_idx >= self.data_len
        info = {}
        return self.get_observation(), reward, done, info


    def render(self):
        # Render the environment
        print(self.data[self.current_data_idx])

    def reset(self):
        # Reset the environment
        self.current_data_idx = 0
        self.balance = 100000
        self.shares = 0
        return self.get_observation()

    def get_observation(self):
        # Get the observation of the environment
        if self.current_data_idx >= 45:
            data_slice = self.data[self.current_data_idx - 45:self.current_data_idx]
            observation = data_slice.astype('float32')
        else:
            observation = np.zeros((45, 9))
        return observation

In [10]:
env=TradingEnv(data)

In [11]:
from stable_baselines3.common.env_checker import check_env


In [12]:
log_path = os.path.join('Training', 'Logs')


In [13]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [14]:
model.learn(total_timesteps=5)

Logging to Training\Logs\PPO_9


IndexError: invalid index to scalar variable.

AttributeError: type object 'TradingEnv' has no attribute 'self'